# Knowledge base System
문서를 이용한 벡터DB 구성 실습

In [1]:
# 원본 tax.docx

# 문서 로딩
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('./tax_pure.docx')
docu = loader.load()

# 문서 분할
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 강사님이 추천해준 값
    chunk_overlap=200, # 강사님이 추천해준 값
)

doc_list = loader.load_and_split(text_splitter=splitter)

# 임베딩
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# 임베딩 모델
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

# 벡터 DB 적재
from langchain_chroma import Chroma
database = Chroma.from_documents(
    documents=doc_list, 
    embedding=embedding,
    collection_name='chroma-tax',
    persist_directory='./chroma_origin' # 저장소 이름 다르게 해
)

# Retrieval
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=5)

# augumentation 
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

# gpt-4o-mini
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [context]만 참조해서 사용자의 질문에 답변해 주세요.[context]에 참조해서 답변을 못하면 못한다고 말해주세요.
[context]
{retrieved_docs}
Questrion : {query}
"""
response = llm.invoke(prompt)
response

AIMessage(content='죄송하지만, 주어진 [context]만으로 연봉 5천만원인 직장인의 정확한 소득세를 계산할 수 없습니다. 소득세는 다양한 공제 요소와 세율에 따라 달라지므로, 보다 구체적인 정보를 기반으로 계산해야 합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 5232, 'total_tokens': 5293, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CDtwvqc2hcp1UKdHdcVgZ843Yjo43', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a41d7b9c-cc73-4875-a395-075b0a74539c-0', usage_metadata={'input_tokens': 5232, 'output_tokens': 61, 'total_tokens': 5293, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
# 표 마크타운으로 수정
# 문서 로딩
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('./tax.docx')
docu = loader.load()

# 문서 분할
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 강사님이 추천해준 값
    chunk_overlap=200, # 강사님이 추천해준 값
)

doc_list = loader.load_and_split(text_splitter=splitter)

# 임베딩
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# 임베딩 모델
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

# 벡터 DB 적재
from langchain_chroma import Chroma
database = Chroma.from_documents(
    documents=doc_list, 
    embedding=embedding,
    collection_name='chroma-tax',
    persist_directory='./chroma_tune' # 저장소 이름 다르게 해
)

# Retrieval
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=5)

# augumentation 
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

# gpt-4o-mini

query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [context]만 참조해서 사용자의 질문에 답변해 주세요.[context]에 참조해서 답변을 못하면 못한다고 말해주세요.
[context]
{retrieved_docs}
Questrion : {query}
"""
response = llm.invoke(prompt)
response

AIMessage(content='연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 우선 종합소득 과세표준에 따라 아래의 세율을 적용합니다.\n\n1. 종합소득 과세표준 구간: 5,000만 원 초과 ~ 8,800만 원 이하\n2. 세율: **624만 원** + (5,000만 원 초과분 × 24%)\n\n여기서 5,000만원 초과분은 0원이므로, 소득세는 다음과 같이 계산됩니다.\n\n- 산출세액 = 624만 원 + (0 × 24%) = 624만 원\n\n따라서, 연봉 5천만원인 직장인의 소득세는 **6,240,000원**입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 170, 'prompt_tokens': 5097, 'total_tokens': 5267, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CDtxGJNGektnXac1AjYlELcpVapxI', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--094679dd-302f-4109-aa38-8287cb352b92-0', usage_metadata={'input_tokens': 5097, 'output_tokens': 170, 'total_tokens': 5267, 'input_token_detail